In [1]:
### Init network Build
import numpy as np
from pathlib import Path
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display
import pandas as pd
from osgeo import gdal
import keras
import time
import tensorflow as tf
print(tf.__version__, 'tf')
print(keras.__version__, 'keras')


2.4.1 tf
2.4.3 keras


In [5]:
#2.4.1 tf
#2.4.3 keras

In [4]:
!nvidia-smi

Sun Feb 28 05:26:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   38C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
from google.cloud import storage
# create a service key json file and point to it here,   I will put a copy of the .json key I am using in slack.
storage_client = storage.Client.from_service_account_json('access_key1.json')  # create a service key json file and point to it here

bucket_name = 'pollutemenot-ai'
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
#download srtm
prefix = 'GEE_images_final2/'
dl_dir = 'image_final2/'
blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
for blob in blobs:
    filename = blob.name.replace('/', '_') 
    blob.download_to_filename(dl_dir + filename)  # Download
    
# download 

In [6]:
prefix = 'GEE_images_final/'
dl_dir = '../sample_data/sample3/srtm/'
blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
for blob in blobs:
    print(blob.name)

GEE_images_final/hires/RBG/1000_LRE-2019-00318-120_RBG_hires.tif
GEE_images_final/hires/RBG/1001_LRE-2019-00359-176_RBG_hires.tif
GEE_images_final/hires/RBG/1002_LRE-2019-00375-146_RBG_hires.tif
GEE_images_final/hires/RBG/1003_LRE-2019-00406-56_RBG_hires.tif
GEE_images_final/hires/RBG/1004_LRE-2019-00426-102_RBG_hires.tif
GEE_images_final/hires/RBG/1005_LRE-2019-00434-102_RBG_hires.tif
GEE_images_final/hires/RBG/1006_LRE-2019-00435-143_RBG_hires.tif
GEE_images_final/hires/RBG/1007_LRE-2019-00469-102_RBG_hires.tif
GEE_images_final/hires/RBG/1008_LRE-2019-00569-176_RBG_hires.tif
GEE_images_final/hires/RBG/1009_LRE-2019-00573-171_RBG_hires.tif
GEE_images_final/hires/RBG/1010_LRE-2019-00598-102_RBG_hires.tif
GEE_images_final/hires/RBG/1011_LRE-2019-00640-102_RBG_hires.tif
GEE_images_final/hires/RBG/1012_LRE-2019-00723-45_RBG_hires.tif
GEE_images_final/hires/RBG/1013_LRE-2019-00847-176_RBG_hires.tif
GEE_images_final/hires/RBG/1014_LRE-2019-00881-102_RBG_hires.tif
GEE_images_final/hires/RBG/

In [2]:
tif_list = [f for f in Path('../sample_data/final_data_temp1/').iterdir() if '.tif' in str(f)]
print(len(tif_list))
#tif_list

5517


In [3]:
import pandas as pd
datapath = "../combined_regular_clean.csv"
df = pd.read_csv(datapath, encoding = "ISO-8859-1")
df.head()#.iloc[99]

,jurisdiction_type,da_number,district,project_name,longitude,latitude,date_issued_or_denied,rha_determination,cwa_determination,rha1,...,cwa2,cwa3,cwa4,cwa5,cwa6,cwa7,cwa8,cwa9,potential_wetland,index
0,RAPANOS,LRB-1983-10120,Buffalo,Trade-A-Yacht (Hibiscus Harbor - Union Springs...,-76.70773,42.85821,06/19/2020,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,RAPANOS,LRB-1985-69031,Buffalo,"POOLEY, MARK A.",-75.85524,43.15230,07/07/2016,0,1,0,...,0,1,0,0,0,0,0,0,0,2
2,RAPANOS,LRB-1986-99614,Buffalo,"Bellamy, Michael (previous: MACKO, JOHN)",-78.04046,42.68911,10/12/2017,0,1,0,...,0,0,0,0,0,0,0,0,0,3
3,RAPANOS,LRB-1990-97632,Buffalo,WESTWOOD COUNTRY CLUB,-78.77134,42.97994,06/28/2016,0,1,0,...,0,1,0,0,0,0,0,0,1,4
4,RAPANOS,LRB-1991-98611,Buffalo,MODERN LANDFILL INCORPORATED,-78.97142,43.21616,03/22/2016,0,1,0,...,0,1,0,1,1,0,0,0,1,5


In [4]:
# investigate files
hires = [f for f in tif_list if 'hires' in str(f)]
lores = [f for f in tif_list if 'lores' in str(f)]

from collections import defaultdict
d = defaultdict(list)

for image in tif_list:
    d[image.name.split('_')[6]].append(image)


print(len(hires))
print(len(lores))
#for i in range(100,110):
#   print(tif_list[i].name.split('_'))

2753
2764


In [5]:
counter = 0
for k, v in d.items():
    print(k)
    print(v)
    counter += 1
    if counter >4: break

SAW-2019-01154
[PosixPath('../sample_data/final_data_temp1/GEE_images_final_lores_srtm_12084_SAW-2019-01154_srtm_lores.tif'), PosixPath('../sample_data/final_data_temp1/GEE_images_final_hires_mndwi_12084_SAW-2019-01154_mndwi_hires.tif'), PosixPath('../sample_data/final_data_temp1/GEE_images_final_hires_srtm_12084_SAW-2019-01154_srtm_hires.tif'), PosixPath('../sample_data/final_data_temp1/GEE_images_final_lores_mndwi_12084_SAW-2019-01154_mndwi_lores.tif')]
SAW-2019-01817
[PosixPath('../sample_data/final_data_temp1/GEE_images_final_lores_srtm_12124_SAW-2019-01817_srtm_lores.tif'), PosixPath('../sample_data/final_data_temp1/GEE_images_final_lores_mndwi_12124_SAW-2019-01817_mndwi_lores.tif'), PosixPath('../sample_data/final_data_temp1/GEE_images_final_hires_mndwi_12124_SAW-2019-01817_mndwi_hires.tif'), PosixPath('../sample_data/final_data_temp1/GEE_images_final_hires_srtm_12124_SAW-2019-01817_srtm_hires.tif')]
LRB-2017-00139
[PosixPath('../sample_data/final_data_temp1/GEE_images_final_hire

In [6]:
def unpack_Tiffs(list_of_tiffs):
    layer_list = []
    for t in list_of_tiffs:
        # open the geotiff
        p = gdal.Open(str(t))
        channels = p.RasterCount
        #print(p.GetDescription())
        
        # iterate through rasters and add each to the list of layer_list
        layer_list += [np.array(p.GetRasterBand(i).ReadAsArray()) for i in range(1,channels+1)] 
        
        
    return np.array(layer_list)


In [7]:
def create_array(tif_image):
    p = gdal.Open(str(tif_image))
    channels = p.RasterCount
        
    # iterate through rasters and add each to the list of layer_list
    layer_list = [np.array(p.GetRasterBand(i).ReadAsArray()) for i in range(1,channels+1)] 
    return np.array(layer_list)

In [8]:
def packager1(tf_list):
    if len(tf_list) != 4: print('ERROR tif list is not correct length')
    hires_mndwi = [f for f in tf_list if 'hires' in f.name and 'mndwi' in f.name][0]
    hires_srtm = [f for f in tf_list if 'hires' in f.name and 'srtm' in f.name][0]
    lores_mndwi = [f for f in tf_list if 'lores' in f.name and 'mndwi' in f.name][0]
    lores_srtm = [f for f in tf_list if 'lores' in f.name and 'srtm' in f.name][0]

    hm = create_array(hires_mndwi)
    #print(np.any(np.isnan(hm)))
    #print(np.min(hm), np.max(hm))
    hs = create_array(hires_srtm)
    #print(np.any(np.isnan(hs)))
    #print(np.min(hs), np.max(hs))
    hs = (hs - hs.mean()) * (.3/hs.std()) +.5
    #print(np.min(hs), np.max(hs))

    lm = create_array(lores_mndwi)
    #print(np.any(np.isnan(lm)))
    #print(np.min(lm), np.max(lm))

    ls = create_array(lores_srtm)
    #print(np.any(np.isnan(ls)))
    #print(np.min(ls), np.max(ls))
    ls = (ls - ls.mean()) * (.3/ls.std()) +.5

    return np.concatenate((hm, hs, hm+hs, lm, ls, lm+ls)).reshape(1,256,256,6)
    
    #print(hires_mndwi, hires_srtm, lores_mndwi, lores_srtm)
    
x = packager1(d.get('SAW-2019-01154'))
x.shape

(1, 256, 256, 6)

In [9]:
ids = list(d.keys())[450:550]
X = []
y = []
for i in ids:
    try:
        arr = packager1(d.get(i))
        if np.any(np.isnan(arr)): 
            print(i)
            continue
    except: continue
    X.append(packager1(d.get(i)))
    y.append(df[df.da_number == i]['cwa_determination'].iloc[0])

SAC-2017-01685
ERROR tif list is not correct length
SAC-2017-01337
SAC-2017-01457
LRB-2014-01078
SAC-2017-01637
ERROR tif list is not correct length
LRB-2005-00781
ERROR tif list is not correct length
ERROR tif list is not correct length
ERROR tif list is not correct length
MVN-2019-00863-XX
LRB-2016-00071
ERROR tif list is not correct length
ERROR tif list is not correct length
ERROR tif list is not correct length
SAW-2019-00996
SAC-2017-01523
SAW-2019-01992
ERROR tif list is not correct length
ERROR tif list is not correct length
MVP-2015-04355-MMJ


In [10]:
print(type(X),type(y))
X = np.concatenate(X)
y = np.array(y)
print(X.shape)
print(len(X), len(y))
print([np.mean(y)])

<class 'list'> <class 'list'>
(85, 256, 256, 6)
85 85
[0.43529411764705883]


In [40]:
image_num = []
image_sizes = []
for key, val in d.items():
    image_num.append(len(val))
    
np.mean(image_num)

4.009447674418604

In [3]:
## develop input splitter
from keras.applications.vgg16 import VGG16
vgg16_model1 = VGG16(include_top=False, weights='imagenet', input_shape=(256,256, 3))
vgg16_model2 = VGG16(include_top=False, weights='imagenet', input_shape=(256,256, 3))
print(type(vgg16_model1))
#vgg16_model1.summary()

<class 'tensorflow.python.keras.engine.functional.Functional'>


In [4]:
test_input = np.random.rand(256,256,3)
test_input = test_input.reshape(1, 256,256, 3)
print(test_input.shape)
vgg16_model1.predict(test_input)

(1, 256, 256, 3)


array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          1.0070252 , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          1.0077714 , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          1.2212641 , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          1.0968785 , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          1.0038235 , 0.        ],
         [0.        , 0.        , 0.02593327, ..., 0.        ,
          1.0227069 , 0.        ]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          1.1060636 , 0.        ],
         [0.01398346, 0.        , 0.        , ..., 0.        ,
          1.049451  , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          1.2429132 , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          1.2108804 , 0. 

In [36]:
for i in range(2):
    s = np.random.rand(256,256,6)[:,:, i*3:3*(i+1)].shape
    print(s)

(256, 256, 3)
(256, 256, 3)


In [6]:
Sequential(vgg16_model1)

NameError: name 'Sequential' is not defined

In [7]:
from keras.layers import Dense, Concatenate, Input, Lambda, Flatten, Softmax
from keras.models import Model, Sequential

num_channels = 6
input = Input(shape=(256,256, num_channels))

branch_outputs = []

# loop for however many branches you want to use.  (each branch needs 3 channel depth)
for i in [0,3]:
    # create looping so that vgg16 model input is created ie 256,256,3
    print(input.shape)
    out = Lambda(lambda x: x[:,:,:,i:i+3])(input)
    print(out.shape)
    
    # Setting up your layers in each branch: (currently each branch is identical architecture.)
    if i == 0: out = Sequential(vgg16_model1)(out)   # use pretrained and loaded vgg16
    else: out = Sequential(vgg16_model1)(out)   # use pretrained and loaded vgg16
    print(out.shape)
    out = Dense(10)(out)
    print(out.shape)
    out = Flatten()(out)
    print(out.shape)
    branch_outputs.append(out)

# Concatenating the branches outputs:
out = Concatenate()(branch_outputs)
print(out.shape)

# Add final dense layers and softmax
out = Dense(10)(out)
print(out.shape)
out = Dense(2)(out)
print(out.shape)
out = Softmax()(out)
print(out.shape)

(None, 256, 256, 6)
(None, 256, 256, 3)
(None, 8, 8, 512)
(None, 8, 8, 10)
(None, 640)
(None, 256, 256, 6)
(None, 256, 256, 3)
(None, 8, 8, 512)
(None, 8, 8, 10)
(None, 640)
(None, 1280)
(None, 10)
(None, 2)
(None, 2)


In [8]:
model = Model(inputs=input, outputs=out)   
model.summary()
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 6) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
sequential (Sequential)         (None, 8, 8, 512)    14714688    lambda[0][0]                     
______________________________________________________________________________________________

In [17]:
predicts = model.predict(X)

In [18]:
predicts

array([[0.52614987, 0.47385013],
       [0.50729644, 0.49270362],
       [0.4020764 , 0.59792364],
       [0.5092877 , 0.49071234],
       [0.558615  , 0.44138497],
       [0.5424467 , 0.4575533 ],
       [0.6158485 , 0.38415152],
       [0.9279001 , 0.07209992],
       [0.91072845, 0.08927153],
       [0.7216913 , 0.27830866],
       [0.69085824, 0.30914176],
       [0.6877592 , 0.31224078],
       [0.49993137, 0.50006866],
       [0.6228172 , 0.37718278],
       [0.79717696, 0.20282304],
       [0.57873434, 0.4212657 ],
       [0.6701145 , 0.32988548],
       [0.4465543 , 0.5534457 ],
       [0.47780797, 0.522192  ],
       [0.6321457 , 0.3678543 ],
       [0.4218403 , 0.5781597 ],
       [0.7781874 , 0.22181265],
       [0.50960356, 0.49039644],
       [0.51452   , 0.48548   ],
       [0.47596624, 0.5240337 ],
       [0.60892016, 0.39107987],
       [0.6190333 , 0.38096675],
       [0.76610214, 0.23389782],
       [0.64370394, 0.35629612],
       [0.46383935, 0.5361606 ],
       [0.

In [15]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

In [19]:
t0 = time.time()
model.fit(X, y, epochs=2, batch_size=4, validation_split = 0.2)
print("Train time:", time.time()-t0)

Epoch 1/2
17/17 [==============================] - 141s 8s/step - loss: 0.4854 - mae: 0.5000 - val_loss: 0.5000 - val_mae: 0.5000
Epoch 2/2
17/17 [==============================] - 147s 9s/step - loss: 0.5000 - mae: 0.5000 - val_loss: 0.5000 - val_mae: 0.5000
Train time: 344.0562000274658


In [16]:
import time
test_input = np.random.rand(256,256,6)
test_input = test_input.reshape(1, 256,256, 6)
print(test_input.shape)
t0 = time.time()
print(model.predict(test_input))
print(f"time taken: {time.time()-t0} seconds")

(1, 256, 256, 6)
[[0.67090887 0.3290911 ]]
time taken: 3.7134952545166016 seconds


In [13]:
test_input.flatten().min()

9.019184943337422e-08

In [11]:
from keras_visualizer import visualizer 
visualizer(model, format='png', view=True)


ValueError: Keras Visualizer: Layer not supported for visualizing

In [7]:
import IPython
print(IPython.sys_info())

{'commit_hash': '2486838d9',
 'commit_source': 'installation',
 'default_encoding': 'ANSI_X3.4-1968',
 'ipython_path': '/usr/local/lib/python3.6/dist-packages/IPython',
 'ipython_version': '7.16.1',
 'os_name': 'posix',
 'platform': 'Linux-4.9.140-tegra-aarch64-with-Ubuntu-18.04-bionic',
 'sys_executable': '/usr/bin/python3',
 'sys_platform': 'linux',
 'sys_version': '3.6.9 (default, Oct  8 2020, 12:12:24) \n[GCC 8.4.0]'}
